# **Compliance GPT with LangChain**

## **Setup**

In [1]:
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

True

## **Config**

In [2]:
from utils.config import get_config
from utils.models import ModelName, get_model

config = get_config()

In [3]:
USER_ID = 'arkan'
CONVERSATION_ID = 'arkan-2021-07-01-01'

## **Define Model**

In [4]:
from utils.models import ModelName, LLMModelName, EmbeddingModelName, get_model

model_name = ModelName.AZURE_OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config, llm_model_name=LLMModelName.GPT_35_TURBO, embedding_model_name=EmbeddingModelName.EMBEDDING_ADA)

## **Load Vector Store Index**

In [5]:
from databases.vector_store import RedisIndexManager

redis = RedisIndexManager(index_name='bi', embed_model=embed_model, config=config, db_id=0)
vector_store = redis.load_vector_index()

c:\Users\acer\miniconda3\envs\chatbot-ocbc\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## **Get Retriever**

In [6]:
from retriever.retriever_bi.retriever_bi import get_retriever_bi
from retriever.retriever_ojk.retriever_ojk import get_retriever_ojk

retriever = get_retriever_bi(vector_store=vector_store, top_n=4, top_k=20, llm_model=llm_model, embed_model=embed_model, config=config)
retriever_ojk = get_retriever_ojk(vector_store=vector_store, top_n=4, top_k=20, llm_model=llm_model, embed_model=embed_model, config=config)

In [8]:
retriever.invoke("What is QRIS maximum transaction nominal?")

TooManyRequestsError: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"

## **Create Chain**

In [ ]:
from constant.bi.prompt import CONTEXTUALIZE_Q_PROMPT_STR, QA_SYSTEM_PROMPT_STR
from databases.chat_store import RedisChatStore
from chain.chain_bi.chain_bi import create_bi_chain
from chain.rag_chain import create_chain_with_chat_history

chat_store = RedisChatStore(k=4, config=config, db_id=1)

chain = create_bi_chain(
    contextualize_q_prompt_str=CONTEXTUALIZE_Q_PROMPT_STR,
    qa_system_prompt_str=QA_SYSTEM_PROMPT_STR,
    retriever=retriever,
    llm_model=llm_model,
    # chat_store=chat_store
)

chain_history = create_chain_with_chat_history(
    final_chain=chain,
    chat_store=chat_store
)

## **Invoke**

#### **With Question, Answer, and Context**

In [ ]:
from chain.rag_chain import get_response

response = get_response(
    chain=chain_history,
    question="What is QRIS maximum transaction nominal?",
    user_id=USER_ID,
    conversation_id=CONVERSATION_ID
)

response

Error in RootListenersTracer.on_chain_end callback: ResponseError("MISCONF Redis is configured to save RDB snapshots, but it's currently unable to persist to disk. Commands that may modify the data set are disabled, because this instance is configured to report errors during writes if RDB snapshotting fails (stop-writes-on-bgsave-error option). Please check the Redis logs for details about the RDB error.")


{'rewrited question': 'What does QRIS stand for?',
 'answer': 'QRIS stands for Quick Response Code Indonesian Standard.\n\nSource: [PERATURAN ANGGOTA DEWAN GUBERNUR NOMOR 21/18/PADG/2019 TENTANG IMPLEMENTASI STANDAR NASIONAL QUICK RESPONSE CODE UNTUK PEMBAYARAN](https://www.bi.go.id/id/publikasi/peraturan/Documents/faq_padg_211819.pdf)',
 'context': '[\n  {\n    "metadata": {\n      "id": "doc:bi:e2f84f2b1e474d83a0a7ecc27cec266d",\n      "file_id": "db344534",\n      "title": "PERATURAN ANGGOTA DEWAN GUBERNUR NOMOR 21/18/PADG/2019 TENTANG IMPLEMENTASI STANDAR NASIONAL QUICK RESPONSE CODE UNTUK PEMBAYARAN.",\n      "file_link": "https://www.bi.go.id/id/publikasi/peraturan/Documents/faq_padg_211819.pdf",\n      "date": "21 Agustus 2019",\n      "type_of_regulation": "Peraturan Anggota Dewan Gubernur",\n      "sector": "Sistem Pembayaran Dan Pengelolaan Uang Rupiah",\n      "standardized_extracted_file_name": "",\n      "standardized_file_name": "padg-21_18_padg_2019-21082019-peraturan_an

In [9]:
print(response['answer'])

QRIS stands for Quick Response Code Indonesian Standard.

Source: [PERATURAN ANGGOTA DEWAN GUBERNUR NOMOR 21/18/PADG/2019 TENTANG IMPLEMENTASI STANDAR NASIONAL QUICK RESPONSE CODE UNTUK PEMBAYARAN](https://www.bi.go.id/id/publikasi/peraturan/Documents/faq_padg_211819.pdf)


In [10]:
print(response["context"])

[
  {
    "metadata": {
      "id": "doc:bi:e2f84f2b1e474d83a0a7ecc27cec266d",
      "file_id": "db344534",
      "title": "PERATURAN ANGGOTA DEWAN GUBERNUR NOMOR 21/18/PADG/2019 TENTANG IMPLEMENTASI STANDAR NASIONAL QUICK RESPONSE CODE UNTUK PEMBAYARAN.",
      "file_link": "https://www.bi.go.id/id/publikasi/peraturan/Documents/faq_padg_211819.pdf",
      "date": "21 Agustus 2019",
      "type_of_regulation": "Peraturan Anggota Dewan Gubernur",
      "sector": "Sistem Pembayaran Dan Pengelolaan Uang Rupiah",
      "standardized_extracted_file_name": "",
      "standardized_file_name": "padg-21_18_padg_2019-21082019-peraturan_anggota_dewan_gubernur_nomor_21_18_padg_2019_tentang_implementasi_standar_nasional_quick_response_code_untuk_pembayaran",
      "page_number": "1",
      "relevance_score": 0.99915075
    },
    "page_content": "StandarNasional QRCodePembayaran (Quick Response Code Indonesian Standard) yang umumnya disingkat sebagai QRIS adalah Standar QRCodePembayaran yang diteta

#### **Streaming**

In [ ]:
from chain.rag_chain import print_answer_stream

print_answer_stream(chain=chain_history, question="Berapa maksimum transaksi QRIS?", user_id=USER_ID, conversation_id=CONVERSATION_ID)

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


Jumlah maksimum transaksi yang dapat dilakukan dengan QRIS adalah Rp5.000.000,00 (lima juta rupiah) per transaksi.

Source: PERATURAN ANGGOTA DEWAN GUBERNUR NOMOR 23/8/PADG/2021 TENTANG PERUBAHAN ATAS PERATURAN ANGGOTA DEWAN GUBERNUR NOMOR 21/18/PADG/2019 TENTANG IMPLEMENTASI STANDAR NASIONAL QUICK RESPONSE CODE UNTUK PEMBAYARAN
[Link to the regulation](https://www.bi.go.id/id/publikasi/peraturan/Documents/FAQ_PADG_230821.pdf#page=2)

#### **Chat Store Monitor**

In [ ]:
# to print chat history
print(chat_store.get_session_history(user_id=USER_ID, conversation_id=CONVERSATION_ID))

Human: Berapa maksimum transaksi QRIS?
AI: Jumlah maksimum transaksi yang dapat dilakukan dengan QRIS adalah Rp5.000.000,00 (lima juta rupiah) per transaksi.

Source: PERATURAN ANGGOTA DEWAN GUBERNUR NOMOR 23/8/PADG/2021 TENTANG PERUBAHAN ATAS PERATURAN ANGGOTA DEWAN GUBERNUR NOMOR 21/18/PADG/2019 TENTANG IMPLEMENTASI STANDAR NASIONAL QUICK RESPONSE CODE UNTUK PEMBAYARAN
[Link to the regulation](https://www.bi.go.id/id/publikasi/peraturan/Documents/FAQ_PADG_230821.pdf#page=2)


In [ ]:
# to make the chat history empty
chat_store.clear_all()